In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [3]:
%run './imports.ipynb'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.3 MB/s eta 0:00:00


In [4]:
def get_ids(folder_path):
    # Get all the files in the folder
    files = os.listdir(folder_path)
    # Extract the IDs from the filenames
    ids = []
    for file in files:
        if file.startswith("shhs1-") and file.endswith(".edf"):
            id_val = file[len("shhs1-"):-len(".edf")]
            try:
                id_val = int(id_val)
                ids.append(id_val)
            except ValueError:
                print(f"Warning: Invalid ID format in filename '{file}'. Skipping.")

    return ids

In [5]:
def crosscorr(x,y):
    correlation = np.correlate(x,y,'full')
    return correlation.argmax() - (len(x) - 1)

In [6]:
def cross_correlation(corr_data):

    df= pd.DataFrame()
    df['delta']= pd.Series(corr_data['Delta'])
    df['theta'] = pd.Series(corr_data['Theta'])
    df['alpha'] = pd.Series(corr_data['Alpha'])
    df['sigma'] = pd.Series(corr_data['Sigma'])
    df['beta'] = pd.Series(corr_data['Beta'])
    df['gamma1'] = pd.Series(corr_data['Gamma1'])
    df['gamma2'] = pd.Series(corr_data['Gamma2'])
    df['gamma2'] = pd.Series(corr_data['Gamma2'])
    df['{}'.format(corr_data.columns[-1])] = pd.Series(corr_data['{}'.format(corr_data.columns[-1])])


    #print(df.corr())
    delta= crosscorr(df['delta'], df['{}'.format(corr_data.columns[-1])])
    theta = crosscorr(df['theta'], df['{}'.format(corr_data.columns[-1])])
    alpha = crosscorr(df['alpha'], df['{}'.format(corr_data.columns[-1])])
    sigma = crosscorr(df['sigma'], df['{}'.format(corr_data.columns[-1])])
    beta =  crosscorr(df['beta'], df['{}'.format(corr_data.columns[-1])])
    gamma1 = crosscorr(df['gamma1'], df['{}'.format(corr_data.columns[-1])])
    gamma2 = crosscorr(df['gamma2'],df['{}'.format(corr_data.columns[-1])])
    lag_values = np.array([delta,theta,alpha,sigma,beta,gamma1,gamma2])

    return lag_values

In [7]:
def lag_execution(input_data):
  #freq_bands = freq_bands.reset_index()
  # Splitting the signal into segments of 60sec
  # 30 sec = 30*125 = 3750 data points
  #window size = 60sec with 30sec overlap
    df = input_data[['Delta', 'Theta', 'Alpha','Sigma', 'Beta', 'Gamma1', 'Gamma2']]
    df_hr=df.copy()
    df_hr['hr']=input_data['hr']
    df_sao2=df.copy()
    df_sao2['sao2']=input_data['sao2']
    df_emg=df.copy()
    df_emg['emg']=input_data['emg']
    df_eog_l=df.copy()
    df_eog_l['eog_l']=input_data['eog_l']
    df_eog_r=df.copy()
    df_eog_r['eog_r']=input_data['eog_r']
    df_airflow=df.copy()
    df_airflow['airflow']=input_data['airflow']

#     freq_data = input_data.reset_index()
    list_comp_data_hr = [df_hr[i:i+60] for i in range(0,len(df_hr),30)]
    #print(list_comp_data_hr)

#   #Pass the window data and compute correlation matrix
    lag_max_hr = pd.DataFrame(columns = ['D_hr','T_hr', 'A_hr', 'S_hr', 'B_hr', 'G1_hr', 'G2_hr'])
#   #corr_max = pd.DataFrame(columns = ['A_B', 'A_S'])

    for i in range(0,len(list_comp_data_hr)):
        lag_hr = cross_correlation(list_comp_data_hr[i])
        # lag_max_hr = lag_max_hr.append(pd.DataFrame(lag_hr.reshape(1,-1), columns=list(lag_max_hr)), ignore_index=True)
        lag_max_hr = pd.concat([lag_max_hr, pd.DataFrame(lag_hr.reshape(1,-1), columns=list(lag_max_hr))], ignore_index=True)


    #SAO2

    list_comp_data_sao2 = [df_sao2[i:i+60] for i in range(0,len(df_sao2),30)]
    lag_max_sao2 = pd.DataFrame(columns = ['D_sao2','T_sao2', 'A_sao2', 'S_sao2', 'B_sao2', 'G1_sao2', 'G2_sao2'])
    for i in range(0,len(list_comp_data_sao2)):
        lag_sao2 = cross_correlation(list_comp_data_sao2[i])
        # lag_max_sao2 = lag_max_sao2.append(pd.DataFrame(lag_sao2.reshape(1,-1), columns=list(lag_max_sao2)), ignore_index=True)
        lag_max_sao2 = pd.concat([lag_max_sao2, pd.DataFrame(lag_sao2.reshape(1,-1), columns=list(lag_max_sao2))], ignore_index=True)
    #emg
    list_comp_data_emg = [df_emg[i:i+60] for i in range(0,len(df_emg),30)]
    lag_max_emg = pd.DataFrame(columns = ['D_emg','T_emg', 'A_emg', 'S_emg', 'B_emg', 'G1_emg', 'G2_emg'])
    for i in range(0,len(list_comp_data_emg)):
        lag_emg = cross_correlation(list_comp_data_emg[i])
        # lag_max_emg = lag_max_emg.append(pd.DataFrame(lag_emg.reshape(1,-1), columns=list(lag_max_emg)), ignore_index=True)
        lag_max_emg = pd.concat([lag_max_emg, pd.DataFrame(lag_emg.reshape(1,-1), columns=list(lag_max_emg))], ignore_index=True)

    #eog_l
    list_comp_data_eog_l = [df_eog_l[i:i+60] for i in range(0,len(df_eog_l),30)]
    lag_max_eog_l = pd.DataFrame(columns = ['D_eog_l','T_eog_l', 'A_eog_l', 'S_eog_l', 'B_eog_l', 'G1_eog_l', 'G2_eog_l'])
    for i in range(0,len(list_comp_data_eog_l)):
        lag_eog_l = cross_correlation(list_comp_data_eog_l[i])
        # lag_max_eog_l = lag_max_eog_l.append(pd.DataFrame(lag_eog_l.reshape(1,-1), columns=list(lag_max_eog_l)), ignore_index=True)
        lag_max_eog_l = pd.concat([lag_max_eog_l, pd.DataFrame(lag_eog_l.reshape(1,-1), columns=list(lag_max_eog_l))], ignore_index=True)

    #eog_R
    list_comp_data_eog_r = [df_eog_r[i:i+60] for i in range(0,len(df_eog_r),30)]
    lag_max_eog_r = pd.DataFrame(columns = ['D_eog_r','T_eog_r', 'A_eog_r', 'S_eog_r', 'B_eog_r', 'G1_eog_r', 'G2_eog_r'])
    for i in range(0,len(list_comp_data_eog_r)):
        lag_eog_r = cross_correlation(list_comp_data_eog_r[i])
        # lag_max_eog_r = lag_max_eog_r.append(pd.DataFrame(lag_eog_r.reshape(1,-1), columns=list(lag_max_eog_r)), ignore_index=True)
        lag_max_eog_r = pd.concat([lag_max_eog_r, pd.DataFrame(lag_eog_r.reshape(1,-1), columns=list(lag_max_eog_r))], ignore_index=True)

    #airflow
    list_comp_data_airflow = [df_airflow[i:i+60] for i in range(0,len(df_airflow),30)]
    lag_max_airflow = pd.DataFrame(columns = ['D_airflow','T_airflow', 'A_airflow', 'S_airflow', 'B_airflow', 'G1_airflow', 'G2_airflow'])
    for i in range(0,len(list_comp_data_airflow)):
        lag_airflow = cross_correlation(list_comp_data_airflow[i])
        # lag_max_airflow = lag_max_airflow.append(pd.DataFrame(lag_airflow.reshape(1,-1), columns=list(lag_max_airflow)), ignore_index=True)
        lag_max_airflow = pd.concat([lag_max_airflow, pd.DataFrame(lag_airflow.reshape(1,-1), columns=list(lag_max_airflow))], ignore_index=True)
    return lag_max_hr,lag_max_sao2, lag_max_emg, lag_max_eog_l, lag_max_eog_r, lag_max_airflow

In [8]:
def stable_signal(arr):
    diff1  = abs(arr[0]-arr[1])
    diff2 = abs(arr[0]-arr[2])
    diff3 = abs(arr[0]-arr[3])
    diff4 = abs(arr[0]-arr[4])
    diff5 = abs(arr[0]-arr[5])


    diff_array = np.array([diff1,diff2,diff3, diff4, diff5])
    #print(diff_array)
    c = np.count_nonzero(diff_array<=1)

    if c >= 4:
        return "stable"
    else:
        return "unstable"

In [9]:
def stability_matrix_execution(lg_max, sig_name):
    stability_matrix1 = pd.DataFrame(columns = ['delta','theta','alpha','sigma','beta','gamma1','gamma2'])
    for i in range (0, len(lg_max)-5):
        stability_D = stable_signal(np.array(lg_max['D_{}'.format(sig_name)][i:i+6]))
        stability_T = stable_signal(np.array(lg_max['T_{}'.format(sig_name)][i:i+6]))
        stability_A = stable_signal(np.array(lg_max['A_{}'.format(sig_name)][i:i+6]))
        stability_S = stable_signal(np.array(lg_max['S_{}'.format(sig_name)][i:i+6]))
        stability_B = stable_signal(np.array(lg_max['B_{}'.format(sig_name)][i:i+6]))
        stability_G1 = stable_signal(np.array(lg_max['G1_{}'.format(sig_name)][i:i+6]))
        stability_G2 = stable_signal(np.array(lg_max['G2_{}'.format(sig_name)][i:i+6]))

        values = np.array([stability_D, stability_T, stability_A, stability_S,stability_B,stability_G1,stability_G2])
        # stability_matrix1 = stability_matrix1.append(pd.DataFrame(values.reshape(1,-1), columns=list(stability_matrix1)), ignore_index=True)
        stability_matrix1 = pd.concat([stability_matrix1, pd.DataFrame(values.reshape(1,-1), columns=list(stability_matrix1))], ignore_index=True)
    return stability_matrix1

In [10]:
def export_TDS(id):
  signal_data = pickle.load(open("./Data/Raw/MSA_male/{}_data.p".format(id),"rb"))
  awake = signal_data.loc[(signal_data['sleep_stage'] == 0)]
  n12 = signal_data.loc[(signal_data['sleep_stage'] == 1) | (signal_data['sleep_stage'] == 2)]
  n34 = signal_data.loc[(signal_data['sleep_stage'] == 3)| (signal_data['sleep_stage'] == 4)]
  rem = signal_data.loc[(signal_data['sleep_stage'] == 5)]
  a_hr, a_sao2,a_emg, a_eog_l, a_eog_r,a_airflow = lag_execution(awake)
  ls_hr, ls_sao2,ls_emg, ls_eog_l, ls_eog_r,ls_airflow = lag_execution(n12)
  ds_hr, ds_sao2,ds_emg, ds_eog_l, ds_eog_r,ds_airflow = lag_execution(n34)
  rem_hr, rem_sao2,rem_emg, rem_eog_l, rem_eog_r,rem_airflow = lag_execution(rem)
  stab_awake_hr = stability_matrix_execution(a_hr, 'hr').apply(pd.value_counts).T
  stab_awake_sao2 = stability_matrix_execution(a_sao2, 'sao2').apply(pd.value_counts).T
  stab_awake_emg = stability_matrix_execution(a_emg, 'emg').apply(pd.value_counts).T
  stab_awake_eog_l = stability_matrix_execution(a_eog_l, 'eog_l').apply(pd.value_counts).T
  stab_awake_eog_r = stability_matrix_execution(a_eog_r, 'eog_r').apply(pd.value_counts).T
  # stab_awake_abdomen = stability_matrix_execution(a_abdomen, 'abdomen').apply(pd.value_counts).T
  stab_awake_airflow = stability_matrix_execution(a_airflow, 'airflow').apply(pd.value_counts).T
  stab_ls_hr = stability_matrix_execution(ls_hr, 'hr').apply(pd.value_counts).T
  stab_ls_sao2 = stability_matrix_execution(ls_sao2, 'sao2').apply(pd.value_counts).T
  stab_ls_emg = stability_matrix_execution(ls_emg, 'emg').apply(pd.value_counts).T
  stab_ls_eog_l = stability_matrix_execution(ls_eog_l, 'eog_l').apply(pd.value_counts).T
  stab_ls_eog_r = stability_matrix_execution(ls_eog_r, 'eog_r').apply(pd.value_counts).T
  # stab_ls_abdomen = stability_matrix_execution(ls_abdomen, 'abdomen').apply(pd.value_counts).T
  stab_ls_airflow = stability_matrix_execution(ls_airflow, 'airflow').apply(pd.value_counts).T
  stab_ds_hr = stability_matrix_execution(ds_hr, 'hr').apply(pd.value_counts).T
  stab_ds_sao2 = stability_matrix_execution(ds_sao2, 'sao2').apply(pd.value_counts).T
  stab_ds_emg = stability_matrix_execution(ds_emg, 'emg').apply(pd.value_counts).T
  stab_ds_eog_l = stability_matrix_execution(ds_eog_l, 'eog_l').apply(pd.value_counts).T
  stab_ds_eog_r = stability_matrix_execution(ds_eog_r, 'eog_r').apply(pd.value_counts).T
  # stab_ds_abdomen = stability_matrix_execution(ds_abdomen, 'abdomen').apply(pd.value_counts).T
  stab_ds_airflow = stability_matrix_execution(ds_airflow, 'airflow').apply(pd.value_counts).T
  stab_rem_hr = stability_matrix_execution(rem_hr, 'hr').apply(pd.value_counts).T
  stab_rem_sao2 = stability_matrix_execution(rem_sao2, 'sao2').apply(pd.value_counts).T
  stab_rem_emg = stability_matrix_execution(rem_emg, 'emg').apply(pd.value_counts).T
  stab_rem_eog_l = stability_matrix_execution(rem_eog_l, 'eog_l').apply(pd.value_counts).T
  stab_rem_eog_r = stability_matrix_execution(rem_eog_r, 'eog_r').apply(pd.value_counts).T
  # stab_rem_abdomen = stability_matrix_execution(rem_abdomen, 'abdomen').apply(pd.value_counts).T
  stab_rem_airflow = stability_matrix_execution(rem_airflow, 'airflow').apply(pd.value_counts).T
  dfs = [stab_awake_hr, stab_awake_sao2, stab_awake_emg, stab_awake_eog_l, stab_awake_eog_r,
       stab_awake_airflow, stab_ls_hr, stab_ls_sao2, stab_ls_emg, stab_ls_eog_l, stab_ls_eog_r,
       stab_ls_airflow, stab_ds_hr, stab_ds_sao2, stab_ds_emg, stab_ds_eog_l, stab_ds_eog_r,
       stab_ds_airflow, stab_rem_hr, stab_rem_sao2, stab_rem_emg, stab_rem_eog_l, stab_rem_eog_r, stab_rem_airflow]
  for df in dfs:
    if 'stable' not in df.columns:
      df = df.assign(stable=0)
    if 'unstable' not in df.columns:
      df = df.assign(unstable=100)
    df.fillna(0, inplace=True)

    try:
      df['score'] = df['stable'] / (df['stable'] + df['unstable'])
      df['score'] = df['score'].apply(lambda x: 0 if x <= 0.45 else 1 if x <= 0.8 else 3)
    except KeyError as e:
      raise KeyError(f"Column 'stable' or 'unstable' not found in DataFrame {df}")

  with pd.ExcelWriter('./Outputs/TDS/MSA_male/{}_c_awake.xlsx'.format(id))as writer:
      stab_awake_hr.reset_index().to_excel(writer, sheet_name = 'hr')
      stab_awake_sao2.reset_index().to_excel(writer, sheet_name = 'sao2')
      stab_awake_emg.reset_index().to_excel(writer, sheet_name = 'emg')
      stab_awake_eog_l.reset_index().to_excel(writer, sheet_name = 'eog_l')
      stab_awake_eog_r.reset_index().to_excel(writer, sheet_name = 'eog_r')
      stab_awake_airflow.reset_index().to_excel(writer, sheet_name = 'airflow')
  with pd.ExcelWriter('./Outputs/TDS/MSA_male/{}_c_ls.xlsx'.format(id))as writer:
      stab_ls_hr.reset_index().to_excel(writer, sheet_name = 'hr')
      stab_ls_sao2.reset_index().to_excel(writer, sheet_name = 'sao2')
      stab_ls_emg.reset_index().to_excel(writer, sheet_name = 'emg')
      stab_ls_eog_l.reset_index().to_excel(writer, sheet_name = 'eog_l')
      stab_ls_eog_r.reset_index().to_excel(writer, sheet_name = 'eog_r')
      # stab_ls_abdomen.reset_index().to_excel(writer, sheet_name = 'abdomen')
      stab_ls_airflow.reset_index().to_excel(writer, sheet_name = 'airflow')
  with pd.ExcelWriter('./Outputs/TDS/MSA_male/{}_c_ds.xlsx'.format(id))as writer:
      stab_ds_hr.reset_index().to_excel(writer, sheet_name = 'hr')
      stab_ds_sao2.reset_index().to_excel(writer, sheet_name = 'sao2')
      stab_ds_emg.reset_index().to_excel(writer, sheet_name = 'emg')
      stab_ds_eog_l.reset_index().to_excel(writer, sheet_name = 'eog_l')
      stab_ds_eog_r.reset_index().to_excel(writer, sheet_name = 'eog_r')
      # stab_ds_abdomen.reset_index().to_excel(writer, sheet_name = 'abdomen')
      stab_ds_airflow.reset_index().to_excel(writer, sheet_name = 'airflow')
  with pd.ExcelWriter('./Outputs/TDS/MSA_male/{}_c_rem.xlsx'.format(id))as writer:
      stab_rem_hr.reset_index().to_excel(writer, sheet_name = 'hr')
      stab_rem_sao2.reset_index().to_excel(writer, sheet_name = 'sao2')
      stab_rem_emg.reset_index().to_excel(writer, sheet_name = 'emg')
      stab_rem_eog_l.reset_index().to_excel(writer, sheet_name = 'eog_l')
      stab_rem_eog_r.reset_index().to_excel(writer, sheet_name = 'eog_r')
      # stab_rem_abdomen.reset_index().to_excel(writer, sheet_name = 'abdomen')
      stab_rem_airflow.reset_index().to_excel(writer, sheet_name = 'airflow')
  return stab_rem_hr

In [11]:
ids = get_ids('./Data/Raw/MSA_male')
# ids = [202232]
for id in ids:
  df = export_TDS(id)
  print(df)

        stable  unstable  score
delta       73       139      0
theta       24       188      0
alpha       65       147      0
sigma       48       164      0
beta        65       147      0
gamma1     117        95      1
gamma2     114        98      1
        unstable  stable  score
delta        154       5      0
theta        153       6      0
alpha        151       8      0
sigma        154       5      0
beta         154       5      0
gamma1       158       1      0
gamma2       149      10      0
        unstable  stable  score
delta         39       2      0
theta         39       2      0
alpha         38       3      0
sigma         38       3      0
beta          40       1      0
gamma1        38       3      0
gamma2        38       3      0
        stable  unstable  score
delta       65        60      1
theta       20       105      0
alpha       25       100      0
sigma       26        99      0
beta        45        80      0
gamma1      66        59      1
gamma2  